In [32]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
print('tensorflow version:', tf.__version__)
print('numpy version:', np.__version__)

tensorflow version: 1.9.0
numpy version: 1.15.0


### Artificial dataset

In [33]:
def generate_data(n_pts=10000, n_features=3, use_nonlinear=False, 
                    noise_std=0.1, train_test_split=4):
    """
    n_pts - number of data points to generate
    n_features - a positive integer - number of features
    use_nonlinear - if True, generate non-linear data
    """
    # Linear data or non-linear data?
    if use_nonlinear:
        weights = np.array([[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]])
    else:
        weights = np.array([1.0, 0.5, 0.2])
          
    bias = np.ones(n_pts).reshape((-1,1))
    low, high = -np.ones((n_pts,n_features),'float'), np.ones((n_pts,n_features),'float')
    np.random.seed(42)
    noise = np.random.normal(size=(n_pts, 1))
    
    np.random.seed(42)
    gen_X = np.random.uniform(low=low, high=high)
    if use_nonlinear:
        gen_y = (weights[0,0] * bias + np.dot(gen_X, weights[0, :]).reshape((-1,1)) + 
             np.dot(gen_X*gen_X, weights[1, :]).reshape([-1,1]) +
             noise_std * noise)
    else:
        gen_y = (weights[0] * bias + np.dot(gen_X, weights[:]).reshape((-1,1)) + 
             noise_std * noise)
    return gen_X, gen_y

In [34]:
# linear data with 3 features
data_X, data_y = generate_data(n_pts=10000, n_features=3, use_nonlinear=False)
data_X.shape, data_y.shape

((10000, 3), (10000, 1))

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    data_X, data_y, test_size=0.25,random_state=714)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(7500, 3) (7500, 1) (2500, 3) (2500, 1)


## Linear Regression with Tensorflow
#### 1. constant inputs

In [5]:
# add column of ones to training data
X_ones = np.c_[X_train, np.ones((len(X_train), 1))]

# define input constant values
X = tf.constant(X_ones, dtype=tf.float32, name='X')
y = tf.constant(y_train, dtype=tf.float32, name='y')

# X transpose for normal equation
XT = tf.transpose(X)

# use normal equation to get model thetas
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[0.997706  ],
       [0.50064737],
       [0.2005714 ],
       [1.0004975 ]], dtype=float32)

#### 2. placeholder with Normal Equation

In [36]:
class LinRegress_NormalEq:
    """
    class LinRegress_NormalEq - implements normal equation solution
    """
    def __init__(self, n_features, learning_rate=0.05, L=0):
        # input placeholders
        self.X = tf.placeholder(tf.float32, [None, n_features], name="X") 
        self.Y = tf.placeholder(tf.float32, [None, 1], name="Y")
    
        # regression parameters for the analytical solution using the Normal equation
        self.theta_in = tf.placeholder(tf.float32, [n_features+1,None])

        # Adding a column of ones to the data matrix
        data_plus_bias = tf.concat([tf.ones([tf.shape(self.X)[0], 1]), self.X], axis=1)
        
        # Normal Equation
        XT = tf.transpose(data_plus_bias)
        self.theta = tf.matmul(tf.matmul(
            tf.matrix_inverse(tf.matmul(XT, data_plus_bias)), XT), self.Y)
        
        # mean square error in terms of theta = theta_in
        self.lr_mse = tf.reduce_mean(tf.square(
            tf.matmul(data_plus_bias, self.theta_in) - self.Y))

In [37]:
model = LinRegress_NormalEq(n_features=X_train.shape[1])
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # get the model thetas
    theta_values = sess.run(model.theta, feed_dict={
        model.X: X_train, model.Y: y_train
    })
    # run the train data for MSE
    lr_mse_train = sess.run(model.lr_mse, feed_dict = {
        model.X: X_train, model.Y: y_train,
        model.theta_in: theta_value
    })
    # run the test data for MSE
    lr_mse_test = sess.run(model.lr_mse, feed_dict = {
        model.X: X_test, model.Y: y_test, 
        model.theta_in: theta_value            
    })

print('train MSE:', lr_mse_train)
print('test MSE:', lr_mse_test)

train MSE: 0.3300895
test MSE: 0.34288436


In [38]:
theta_values

array([[1.0004976 ],
       [0.997706  ],
       [0.50064737],
       [0.20057149]], dtype=float32)

#### 3. Placeholder with Maximum Likelihood Estimator

In [20]:
class LinRegress_MLE:
    """
    class LinRegress_MLE - implements Maximum likelihood Estimate (MLE)
    """
    def __init__(self, n_features, learning_rate=0.05, L=0):
        import math as m
        # input placeholders
        self.X = tf.placeholder(tf.float32, [None, n_features], name="X") 
        self.Y = tf.placeholder(tf.float32, [None, 1], name="Y")
    
        # regression parameters for the analytical solution using the Normal equation
        self.theta_in = tf.placeholder(tf.float32, [n_features+1,None])

        # Adding a column of ones to the data matrix
        data_plus_bias = tf.concat([tf.ones([tf.shape(self.X)[0], 1]), self.X], axis=1)
        
        # Estimate the model using the Maximum Likelihood Estimation (MLE)
        # n_features+2: intercept and std of noise 
        self.weights = tf.Variable(tf.random_normal([n_features+2, 1]))
        
        # prediction from the model
        self.output = tf.matmul(data_plus_bias, self.weights[:-1, :])

        gauss = tf.distributions.Normal(loc=0.0, scale=1.0)
        sigma = 0.0001 + tf.square(self.weights[-1]) 
        pi = tf.constant(m.pi)
        log_LL = tf.log(0.00001 + (1/( tf.sqrt(2*pi)*sigma)) * gauss.prob((self.Y - self.output) / sigma ))  
        
        self.loss = - tf.reduce_mean(log_LL)
        self.train_step = (tf.train.AdamOptimizer(learning_rate).minimize(self.loss), -self.loss)

In [28]:
num_iter = 100
model = LinRegress_MLE(n_features=X_train.shape[1])
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for _ in range(num_iter):
        (_ , loss), weights = sess.run((model.train_step, model.weights), feed_dict={
            model.X: X_train, model.Y: y_train
            })
    # after iteration, perform predictions
    Y_test_predicted = sess.run(model.output, feed_dict={model.X: X_test})

weights

array([[1.001132  ],
       [0.99868095],
       [0.50135475],
       [0.20495257],
       [0.31846252]], dtype=float32)

In [26]:
# from sklearn.metrics import mean_squared_error
# mean_squared_error(y_true=y_test, y_pred=Y_test_predicted)

# from sklearn.metrics import r2_score
# r2_score(y_true=y_test, y_pred=Y_test_predicted)

0.22869110739297585

### Linear Regression with Neural Network